In [17]:
import pandas as pd
import numpy as np
import quandl
from datetime import datetime, timedelta
import json


from dateutil.parser import parse
from datetime import datetime 

In [3]:
# pull in cleaned dataset

data = pd.read_csv('data/combined_clean.csv', low_memory=False)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27492 entries, 0 to 27491
Data columns (total 45 columns):
Unnamed: 0              27492 non-null int64
ticker_symbol           27492 non-null object
co_name                 27492 non-null object
unique_earnings_code    27492 non-null object
factset_sector_num      27492 non-null float64
factset_ind_num         27492 non-null float64
calendar_qtr            27492 non-null object
fiscal_qtr              27492 non-null object
adtv_prev_month         27492 non-null object
report_date             27492 non-null object
eps_est                 27492 non-null object
eps_actual              27492 non-null object
surp_amt                27492 non-null object
rtn_t+3                 27492 non-null float64
mkt_t+3_rtn             27492 non-null float64
rel_t+3_rtn             27492 non-null float64
num_ests_qtr_end        27492 non-null float64
t-7_high_est            27492 non-null float64
t-7_low_est             27492 non-null float64
est_spread

In [ ]:
def add_vol_slope(df):
    """docstring"""
    
    # quandl authentication
    quandl.ApiConfig.api_key = "jUKZLy5xi7gGFf3sSF-r"
    
    

In [10]:
quandl.ApiConfig.api_key = "jUKZLy5xi7gGFf3sSF-r"

In [8]:
toy = data.iloc[:25,:]

In [9]:
toy

,Unnamed: 0,ticker_symbol,co_name,unique_earnings_code,factset_sector_num,factset_ind_num,calendar_qtr,fiscal_qtr,adtv_prev_month,report_date,...,si_days_1m_delta_F,si_days_3m_delta_F,si_pct_1m_delta_F,si_pct_3m_delta_F,3m_delta_an_rtg_F,pre_rpt_rtn_to_tgt_F,3m_delta_tgt_px_F,1m_est_delta_F,3m_est_delta_F,proj_2yr_rev_gr_F
0,0,RAMP,"LiveRamp Holdings, Inc.",RAMP.1Q14,3300.0,3305.0,1Q14,2013/4F,14.4,05/15/2014,...,-0.144062,0.496136,0.057002,0.172538,0.15,33.83,4.44,1.266,7.527,-2.946171
1,2,TCO,"Taubman Centers, Inc.",TCO.1Q14,4800.0,4890.0,1Q14,2014/1F,37.3,04/25/2014,...,0.352543,0.223015,0.171640,0.150831,0.00,2.49,-0.10,-2.722,-3.706,-2.351877
2,5,SNX,SYNNEX Corporation,SNX.1Q14,3250.0,3265.0,1Q14,2014/1F,17.4,04/04/2014,...,0.000000,0.457639,0.069347,0.973769,-0.20,15.84,-5.39,1.198,-7.997,20.412552
3,7,AIV,Apartment Investment and Management Company Cl...,AIV.1Q14,4800.0,4890.0,1Q14,2014/1F,62.7,05/02/2014,...,0.582350,0.547657,-0.160877,-0.058043,0.07,2.13,-2.40,43.314,-3.597,-5.047765
4,10,IT,"Gartner, Inc.",IT.1Q14,3200.0,3205.0,1Q14,2014/1F,51.1,05/01/2014,...,0.015320,-0.107287,-0.218061,-0.025564,0.04,7.41,0.97,-0.270,-22.105,21.560045
5,12,MPWR,"Monolithic Power Systems, Inc.",MPWR.1Q14,1300.0,1305.0,1Q14,2014/1F,23.0,04/25/2014,...,-0.216623,-0.166703,0.029595,-0.058553,-0.02,10.10,-13.44,-0.475,-6.250,26.303779
6,14,WETF,"WisdomTree Investments, Inc.",WETF.1Q14,4800.0,4845.0,1Q14,2014/1F,39.2,05/02/2014,...,-0.163667,0.697238,-0.043391,0.779130,-0.09,42.81,9.57,-3.379,-7.241,113.565568
7,15,SLG,SL Green Realty Corp.,SLG.1Q14,4800.0,4890.0,1Q14,2014/1F,58.5,04/24/2014,...,0.205259,0.479865,0.051846,0.014484,0.03,-0.41,-2.20,16.468,6.833,-3.442242
8,16,SKX,"Skechers U.S.A., Inc. Class A",SKX.1Q14,2400.0,2440.0,1Q14,2014/1F,41.2,04/23/2014,...,0.822023,-0.319613,-0.104418,-0.466693,0.21,12.92,-8.86,1.005,8.065,34.137421
9,17,BNNY,"Annie's, Inc.",BNNY.1Q14,2400.0,2410.0,1Q14,2013/4F,12.9,05/30/2014,...,-0.066202,0.488647,-0.227363,-0.197812,-0.11,30.82,-2.33,-0.016,-0.208,44.404971


In [18]:
dates = list(toy['report_date'])

In [19]:
dates[0]

'05/15/2014'

In [26]:
dt = datetime.strptime(dates[0], '%m/%d/%Y')
str(dt)

'2014-05-15 00:00:00'

In [27]:
new_dates = []

for d in dates:
    dt = datetime.strptime(d, '%m/%d/%Y')
    reformatted = str(dt)
    reformatted = reformatted[:10]
    new_dates.append(reformatted)





In [28]:
new_dates

['2014-05-15',
 '2014-04-25',
 '2014-04-04',
 '2014-05-02',
 '2014-05-01',
 '2014-04-25',
 '2014-05-02',
 '2014-04-24',
 '2014-04-23',
 '2014-05-30',
 '2014-05-09',
 '2014-05-07',
 '2014-05-13',
 '2014-05-07',
 '2014-06-03',
 '2014-05-01',
 '2014-04-30',
 '2014-05-02',
 '2014-04-29',
 '2014-05-14',
 '2014-05-07',
 '2014-04-30',
 '2014-04-24',
 '2014-05-07',
 '2014-04-23']